In [2]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import df2img
import os


In [3]:
# Load the .gpkg file
gdf = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\community\assessorparcels_20210708_closedroll.gpkg"")


# Count unique values in the 'landuse' column
landuse_counts = gdf['LandUse'].value_counts()

print(landuse_counts)



LandUse
SINGLE FAMILY RESIDENCE          20779
CONDOS,COMMUNITY APT PROJS        6443
MOBILE HOMES                      2526
VACANT                            1494
RESIDENTIAL INCOME, 2-4 UNITS     1070
                                 ...  
BOWLING ALLEYS                       1
HEAVY INDUSTRY                       1
POULTRY                              1
SHOPPING CENTERS (REGIONAL)          1
DRIVE-IN THEATRES                    1
Name: count, Length: 82, dtype: int64


In [4]:

landuse_counts.sum()


37205

In [5]:

# Convert the Series to a DataFrame for plotting
df = landuse_counts.reset_index()
df.columns = ['LandUse', 'Count']


In [ ]:


%
# Map categories to the land use types
categories = {
    'Critical Infrastructure': ['UTILITY,WATER COMPANY', 'LIGHT MANUFACTURING', 'RIGHTS OF WAY,SEWER,LAND FILLS,ETC',
                                'WATER RIGHTS,PUMPS', 'PUBLIC BLDGS,FIREHOUSES,MUSEUMS,POST OFFICES,ETC', 'SCHOOLS', 
                                'HOSPITALS', 'PIPELINES,CANALS', 'PETROLEUM AND GAS','WASTE', 'HIGHWAYS AND STREETS', 'COLLEGES'],
    'Residential': ['SINGLE FAMILY RESIDENCE', 'CONDOS,COMMUNITY APT PROJS', 'MOBILE HOMES', 'RESIDENTIAL INCOME, 2-4 UNITS',
                    'RANCHO ESTATES (RURAL HOME SITES)', 'APARTMENTS, 5 OR MORE UNITS', 'MIXED USE-COMMERCIAL/RESIDENTIAL','MOBILE HOME PARKS'],
    'Commercial': ['RETAIL STORES, SINGLE STORY', 'COMMERCIAL (MISC)', 'COMMERCIAL AND OFFICE CONDOS,PUDS',
                   'OFFICE BUILDINGS, SINGLE STORY', 'OFFICE BUILDINGS, MULTI-STORY', 'AUTO SALES, REPAIR, STORAGE, CAR WASH, ETC',
                   'RESTAURANTS,BARS', 'HOTELS', 'SERVICE STATIONS', 'STORE AND OFFICE COMBINATION', 'SHOPPING CENTERS (NEIGHBORHOOD)',
                   'BANKS, S&LS', 'SUPERMARKETS', 'WAREHOUSING',  'INDUSTRIAL, MISC','PACKING PLANTS'],
    'Agricultural': ['PASTURE OF GRAZING, DRY', 'ORCHARDS, IRRIGATED', 'IRRIGATED FARMS, MISC', 'FIELD CROPS-IRRIGATED','FIELD CROPS, DRY',
                     'VINES AND BUSH FRUIT-IRRIGATED', 'DRY FARMS (MISC)', 'PASTURE-IRRIGATED', 'VINES AND BUSH FRUIT-IRRIGATED', 'NURSERIES,GREENHOUSES', 'ORCHARDS', 'GOLF COURSES'],
    'Miscellaneous': ['VACANT', 'RECREATIONAL OPEN (MISC)', 'MISCELLANEOUS', 'BEACHES, SAND DUNES',
                      'PARKS',  'INSTITUTIONAL (MISC)', 
                      'RIVERS AND LAKES', 'FLOWERS', 'INDUSTRIAL CONDOS,PUDS', 'OPEN STORAGE, BULK PLANT',
                      'CLUBS, LODGE HALLS', 'PROFESSIONAL BUILDINGS', 'MORTUARIES,CEMETERIES,MAUSOLEUMS', 
                      'CAMPS, CABINS', 'REST HOMES', 'HORSES', 'RACE TRACKS, RIDING STABLES', 'WHOLESALE LAUNDRY', 'LUMBER YARDS, MILLS',
                      'DAY CARE', 'RECREATION',  'DANCE HALLS', 'OTHER FOOD PROCESSING, BAKERIES',
                      'BED AND BREAKFAST', 'TRUCK CROPS-IRRIGATED', 'VINEYARDS', 'AUDITORIUMS, STADIUMS', 'TREE FARMS', 'BOWLING ALLEYS',
                      'HEAVY INDUSTRY', 'POULTRY', 'SHOPPING CENTERS (REGIONAL)', 'DRIVE-IN THEATRES', 'CHURCHES, RECTORY', 'PARKING LOTS']
}

Critical Infrastructure: These are facilities and systems that are essential for the functioning of a community and its safety during a wildfire event. They include:
Emergency response facilities (firehouses, hospitals, etc.)
Essential utility systems (water, power, communication)
Transportation networks (highways, streets)
Community support facilities (schools, colleges, churches, daycare, rest homes)
Protecting and maintaining the operation of critical infrastructure during a wildfire is crucial for effective emergency response, public safety, and community resilience.
High-Risk Infrastructure: These are facilities and structures that pose a higher risk during a wildfire event due to their nature, contents, or operations. They include:
Industrial facilities (manufacturing, warehousing, processing plants)
Facilities handling hazardous materials (petroleum, gas, chemicals)
Structures with high fuel loads (lumber yards, industrial parks)
High-risk infrastructure requires special attention and planning to mitigate potential hazards during a wildfire. These facilities may need additional fire protection measures, emergency shut-off systems, or specialized evacuation procedures to prevent accidents or explosions that could compound the wildfire emergency.

In [6]:

categories = { 'Critical Infrastructure': ['UTILITY,WATER COMPANY', 'WATER RIGHTS,PUMPS', 
                                           'PUBLIC BLDGS,FIREHOUSES,MUSEUMS,POST OFFICES,ETC', 'SCHOOLS', 
                                           'HOSPITALS', 'PIPELINES,CANALS', 
                                           'HIGHWAYS AND STREETS', 'COLLEGES', 
                                           'CHURCHES, RECTORY', 'DAY CARE', 'REST HOMES'], 
              'High-Risk Infrastructure': ['PETROLEUM AND GAS', 'LIGHT MANUFACTURING', 'HEAVY INDUSTRY', 
                                           'WAREHOUSING', 'INDUSTRIAL, MISC', 
                                           'PACKING PLANTS', 'LUMBER YARDS, MILLS', 
                                           'OTHER FOOD PROCESSING, BAKERIES', 'INDUSTRIAL CONDOS,PUDS'], 
              'Residential': ['SINGLE FAMILY RESIDENCE', 'CONDOS,COMMUNITY APT PROJS', 
                              'MOBILE HOMES', 'RESIDENTIAL INCOME, 2-4 UNITS', 
                              'RANCHO ESTATES (RURAL HOME SITES)', 
                              'APARTMENTS, 5 OR MORE UNITS', 
                              'MIXED USE-COMMERCIAL/RESIDENTIAL', 'MOBILE HOME PARKS'], 
              'Commercial': ['RETAIL STORES, SINGLE STORY', 'COMMERCIAL (MISC)', 'COMMERCIAL AND OFFICE CONDOS,PUDS', 
                             'OFFICE BUILDINGS, SINGLE STORY', 'OFFICE BUILDINGS, MULTI-STORY', 
                             'AUTO SALES, REPAIR, STORAGE, CAR WASH, ETC', 'RESTAURANTS,BARS', 'HOTELS', 
                             'SERVICE STATIONS', 'STORE AND OFFICE COMBINATION', 'SHOPPING CENTERS (NEIGHBORHOOD)', 
                             'BANKS, S&LS', 'SUPERMARKETS', 'PROFESSIONAL BUILDINGS', 'WHOLESALE LAUNDRY', 'SHOPPING CENTERS (REGIONAL)'], 
              'Agricultural': ['PASTURE OF GRAZING, DRY', 'ORCHARDS, IRRIGATED', 'IRRIGATED FARMS, MISC', 
                               'FIELD CROPS-IRRIGATED', 'FIELD CROPS, DRY', 'VINES AND BUSH FRUIT-IRRIGATED', 
                               'DRY FARMS (MISC)', 'PASTURE-IRRIGATED', 'NURSERIES,GREENHOUSES', 'ORCHARDS', 'GOLF COURSES', 
                               'VINEYARDS', 'TREE FARMS', 'TRUCK CROPS-IRRIGATED', 'POULTRY'],
            'Miscellaneous': ['VACANT', 'RECREATIONAL OPEN (MISC)', 'MISCELLANEOUS', 'BEACHES, SAND DUNES', 
                              'PARKS', 'INSTITUTIONAL (MISC)', 'RIVERS AND LAKES', 'FLOWERS', 'OPEN STORAGE, BULK PLANT', 
                              'CLUBS, LODGE HALLS', 'MORTUARIES,CEMETERIES,MAUSOLEUMS', 'CAMPS, CABINS', 
                              'HORSES', 'RACE TRACKS, RIDING STABLES', 
                              'RECREATION', 'DANCE HALLS', 'BED AND BREAKFAST', 
                              'AUDITORIUMS, STADIUMS', 'BOWLING ALLEYS', 
                              'DRIVE-IN THEATRES', 'PARKING LOTS', 'RIGHTS OF WAY,SEWER,LAND FILLS,ETC', 'WASTE'] }

In [ ]:
categories = {
    'Critical Infrastructure': ['UTILITY,WATER COMPANY', 'WATER RIGHTS,PUMPS', 
                                'PUBLIC BLDGS,FIREHOUSES,MUSEUMS,POST OFFICES,ETC', 'SCHOOLS', 
                                'HOSPITALS', 'PIPELINES,CANALS', 
                                'HIGHWAYS AND STREETS', 'COLLEGES', 
                                'CHURCHES, RECTORY', 'DAY CARE', 'REST HOMES'],
    'High-Risk Infrastructure': ['PETROLEUM AND GAS', 'LIGHT MANUFACTURING', 'HEAVY INDUSTRY', 
                                 'WAREHOUSING', 'INDUSTRIAL, MISC', 
                                 'PACKING PLANTS', 'LUMBER YARDS, MILLS', 
                                 'OTHER FOOD PROCESSING, BAKERIES', 'INDUSTRIAL CONDOS,PUDS'],
    'Residential': ['SINGLE FAMILY RESIDENCE', 'CONDOS,COMMUNITY APT PROJS', 
                    'MOBILE HOMES', 'RESIDENTIAL INCOME, 2-4 UNITS', 
                    'RANCHO ESTATES (RURAL HOME SITES)', 
                    'APARTMENTS, 5 OR MORE UNITS', 
                    'MIXED USE-COMMERCIAL/RESIDENTIAL', 'MOBILE HOME PARKS'],
    'Commercial': ['RETAIL STORES, SINGLE STORY', 'COMMERCIAL (MISC)', 'COMMERCIAL AND OFFICE CONDOS,PUDS', 
                   'OFFICE BUILDINGS, SINGLE STORY', 'OFFICE BUILDINGS, MULTI-STORY', 
                   'AUTO SALES, REPAIR, STORAGE, CAR WASH, ETC', 'RESTAURANTS,BARS', 'HOTELS', 
                   'SERVICE STATIONS', 'STORE AND OFFICE COMBINATION', 'SHOPPING CENTERS (NEIGHBORHOOD)', 
                   'BANKS, S&LS', 'SUPERMARKETS', 'PROFESSIONAL BUILDINGS', 'WHOLESALE LAUNDRY', 'SHOPPING CENTERS (REGIONAL)'],
    'Agricultural': ['PASTURE OF GRAZING, DRY', 'ORCHARDS, IRRIGATED', 'IRRIGATED FARMS, MISC', 
                     'FIELD CROPS-IRRIGATED', 'FIELD CROPS, DRY', 'VINES AND BUSH FRUIT-IRRIGATED', 
                     'DRY FARMS (MISC)', 'PASTURE-IRRIGATED', 'NURSERIES,GREENHOUSES', 'ORCHARDS', 
                     'VINEYARDS', 'TREE FARMS', 'TRUCK CROPS-IRRIGATED', 'POULTRY',
                     'GOLF COURSES', 'HORSES', 'RACE TRACKS, RIDING STABLES'],
    'Natural Areas': ['BEACHES, SAND DUNES', 'PARKS', 'RIVERS AND LAKES', 'RECREATIONAL OPEN (MISC)'],
    'Miscellaneous': ['VACANT', 'MISCELLANEOUS', 'INSTITUTIONAL (MISC)', 'FLOWERS', 'OPEN STORAGE, BULK PLANT', 
                      'CLUBS, LODGE HALLS', 'MORTUARIES,CEMETERIES,MAUSOLEUMS', 'CAMPS, CABINS', 
                      'RECREATION', 'DANCE HALLS', 'BED AND BREAKFAST', 
                      'AUDITORIUMS, STADIUMS', 'BOWLING ALLEYS', 
                      'DRIVE-IN THEATRES', 'PARKING LOTS', 'RIGHTS OF WAY,SEWER,LAND FILLS,ETC', 'WASTE']
}

In [7]:
# Assign categories to each land use type
def assign_category(landuse):
    for category, types in categories.items():
        if landuse in types:
            return category
    return 'Null'



In [8]:
df['Category'] = df['LandUse'].apply(assign_category)
df


,LandUse,Count,Category
0,SINGLE FAMILY RESIDENCE,20779,Residential
1,"CONDOS,COMMUNITY APT PROJS",6443,Residential
2,MOBILE HOMES,2526,Residential
3,VACANT,1494,Miscellaneous
4,"RESIDENTIAL INCOME, 2-4 UNITS",1070,Residential
...,...,...,...
77,BOWLING ALLEYS,1,Miscellaneous
78,HEAVY INDUSTRY,1,High-Risk Infrastructure
79,POULTRY,1,Agricultural
80,SHOPPING CENTERS (REGIONAL),1,Commercial


In [9]:
# Apply the function to create a new category column in the original GeoDataFrame
gdf['Category'] = gdf['LandUse'].apply(assign_category)

In [13]:
gdf = gdf[['Category', 'geometry']]
gdf

,Category,geometry
0,Residential,"MULTIPOLYGON (((5990542.208 1984356.501, 59905..."
1,Residential,"MULTIPOLYGON (((5990541.829 1984323.803, 59905..."
2,Residential,"MULTIPOLYGON (((5990529.493 1984304.879, 59905..."
3,Residential,"MULTIPOLYGON (((5990312.784 1984314.752, 59903..."
4,Residential,"MULTIPOLYGON (((5990281.658 1984253.979, 59902..."
...,...,...
37610,Residential,"MULTIPOLYGON (((6018829.630 2024824.862, 60190..."
37611,Residential,"MULTIPOLYGON (((6019078.334 2024995.500, 60191..."
37612,Residential,"MULTIPOLYGON (((6019011.436 2024842.907, 60188..."
37613,Agricultural,"MULTIPOLYGON (((6028879.255 2027983.456, 60292..."


In [16]:
output_dir = os.path.join(r"C:\Users\bsf31\Documents\post-meds\data\signal\Moore\built_model\base")
os.makedirs(output_dir, exist_ok=True)

# Specify the output file name and format
output_file = os.path.join(output_dir, "lu_catagories.gpkg")

# Save the GeoDataFrame to the specified file
gdf.to_file(output_file, driver="GPKG")